In [10]:
import pandas as pd
import uuid
import textdistance
from datetime import datetime
dups = pd.read_csv('Fragebogen_with_dups.csv')
dups_types = pd.read_csv('labels_type_integrated.csv')
dups.drop(['Unnamed: 0','patienten_id_doc','blocker'],axis=1,inplace=True)
dups_types.drop(['Unnamed: 0','patienten_id_doc'],axis=1,inplace=True)

In [11]:
def generate_unique_value(row, length):
    if row == ('nan' * length):
        return str(uuid.uuid4())
    return row

def sorted_neighborhood_g(dataframe: pd.DataFrame, types, target_efficiency=1, initial_window_size=3):
    blockers = [col for col in dups.columns if col != 'source']
    dataframe['blocker'] = (
        dataframe[blockers]
        .astype(str)
        .agg(lambda row: ''.join(val[0] for val in row), axis=1)  # Take first character
        .str.lower()
        .str.replace(' ', '', regex=True)
        .apply(generate_unique_value, args=(len(blockers),))
    )
    num_rows = len(dataframe)
    dataframe_sorted = dataframe.sort_values(by='blocker')
    duplicate_pairs = []
    comparisons = 0
    duplicates_found = 0
    window_size = initial_window_size
    for i in range(num_rows):
        print(i,' window size : ', window_size)
        window_end = min(i + window_size, num_rows)
        for j in range(i + 1, window_end):
            comparisons += 1
            comp = compare(dataframe_sorted,i,j,types)
            
            match =  comp > 0.95
            if match:
                print('dup pairs : ',i, ' and ',j,' with sim : ',comp)
                duplicate_pairs.append((i, j))
                duplicates_found += 1
        if comparisons > 0:
            efficiency = duplicates_found / comparisons
            if efficiency > target_efficiency:
                window_size += 1
            else:
                window_size = initial_window_size
                comparisons = 0
                duplicates_found = 0
    dataframe_sorted.drop('blocker', axis=1, inplace=True)
    return duplicate_pairs, dataframe_sorted

In [12]:
def convert_to_seconds(date_list):
    seconds_list = []

    for i, date_str in enumerate(date_list):
        try:
            # Parse the date string into a datetime object
            dt_object = datetime.strptime(date_str, "%Y-%m-%d")

            # Convert datetime object to seconds since the epoch
            seconds = int(dt_object.timestamp())
            seconds_list.append(seconds)
        except Exception as e:
            if i > 0:
                seconds_list.append(seconds_list[-1])
            else:
                # If the first element fails, append None or any placeholder value
                seconds_list.append(None)

    return seconds_list

def convert_to_seconds_str(date_str):
    try:
        dt_object = datetime.strptime(date_str, "%Y-%m-%d")
        seconds = int(dt_object.timestamp())
        return seconds
    except Exception as e:
        return 0
def compare(df, index_i, index_j, types_dict):
    total_similarity = 0
    count = 0
    all_same_as_str = True
    for i in [col for col in df.columns if col != 'blocker' and col != 'source']:
        value_i = str(df.iloc[index_i][i])
        value_j = str(df.iloc[index_j][i])
        if value_i != value_j:
            all_same_as_str = False
            break
    if all_same_as_str:
        return 1.0
    for i in [col for col in df.columns if col != 'blocker' and col != 'source']:
        value_i = df.iloc[index_i][i]
        value_j = df.iloc[index_j][i]
        if pd.isna(value_i) and pd.isna(value_j):
            similarity = 0
        else:
            if types_dict[i] == 'F3':
                similarity = 1 if value_i == value_j else 0
            elif types_dict[i] == 'F8':
                if pd.isna(value_i) or pd.isna(value_j):
                    similarity = 0
                else:
                    col_floats = pd.to_numeric(df[i], errors='coerce').tolist()
                    value_i = float(value_i)
                    value_j = float(value_j)
                    min_val = min(col_floats)
                    max_val = max(col_floats)
                    d_max = max_val - min_val
                    if d_max == 0:
                        similarity = 1 if value_i == value_j else 0
                    else:
                        similarity = 1 - abs(value_i - value_j) / d_max
            elif types_dict[i] == 'A1000':
                if pd.isna(value_i) or pd.isna(value_j):
                    similarity = 0
                else:
                    similarity = textdistance.jaro_winkler(str(value_i), str(value_j))
            else:
                if pd.isna(value_i) or pd.isna(value_j):
                    similarity = 0
                else:
                    col_seconds = convert_to_seconds(list(df[i].dropna()))
                    min_seconds = min(col_seconds)
                    max_seconds = max(col_seconds)
                    d_max = max_seconds - min_seconds
                    value_i_seconds = convert_to_seconds_str(value_i)
                    value_j_seconds = convert_to_seconds_str(value_j)
                    if d_max == 0:
                        similarity = 1 if value_i_seconds == value_j_seconds else 0
                    else:
                        similarity = 1 - abs(value_i_seconds - value_j_seconds) / d_max
            count += 1
        total_similarity += similarity
    return total_similarity / count if count > 0 else 0

In [13]:
dd = pd.read_csv(r'C:\Users\21260\Downloads\union_df.csv', low_memory=False)
dt = pd.read_csv(r'C:\Users\21260\Downloads\non_matched_types.csv', low_memory=False)
dd.drop(['Unnamed: 0','patienten_id_doc'],axis=1, inplace=True)
dd.drop_duplicates(subset=dd.columns[dd.columns != 'source'])

,long_covid_post_covid_patient_timestamp,alter,groe_e,geschlecht,impfung_01_wirkstoff,impfung_02_wirkstoff,impfung_03_wirkstoff,impfung_04_wirkstoff,impfung_01_charge,impfung_02_charge,...,gpcr___4,gpcr___5,gpcr___6,gpcr___7,gpcr___8,gpcr___9,gpcr___10,gpcr___11,gpcr___12,gpcr___13
0,2024-09-02 15:27:28,57.0,175.0,Weiblich,Biontech/Pfizer,Biontech/Pfizer,Moderna,NaN,NaN,NaN,...,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked
1,NaN,56.0,162.0,Weiblich,AstraZeneca,Moderna,Moderna,NaN,ABV8856,3061944,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
2,NaN,42.0,175.0,Männlich,Moderna,Moderna,Moderna,NaN,3001944,3002913,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked
3,2024-09-02 15:27:39,61.0,162.0,Weiblich,NaN,NaN,NaN,NaN,NaN,NaN,...,Checked,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
4,NaN,48.0,0.0,Weiblich,Biontech/Pfizer,Biontech/Pfizer,Moderna,NaN,NaN,NaN,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5142,2022-09-18 21:46:01,39.0,175.0,männlich,AstraZeneca,Moderna,NaN,NaN,ABV4678,3002334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5143,2022-09-19 14:08:00,69.0,175.0,männlich,AstraZeneca,BionTech/ Pfizer,BionTech/ Pfizer,NaN,ABW0018,1C007A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5144,2022-09-19 19:24:54,39.0,165.0,weiblich,BionTech/ Pfizer,BionTech/ Pfizer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5145,2022-09-19 21:06:41,39.0,175.0,weiblich,BionTech/ Pfizer,BionTech/ Pfizer,BionTech/ Pfizer,NaN,EY2172,FA5833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
import re
def clean_string(input_string):
    cleaned_string = re.sub(r'[^\w\s]', ' ', input_string) 
    cleaned_string = ' '.join(cleaned_string.split())
    return cleaned_string.strip()

def dice_similarity(str1, str2):
    str1 = clean_string(str1)
    str2 = clean_string(str2)
    set1 = set(str1.lower().split(' '))
    set2 = set(str2.lower().split(' '))
    intersection = len(set1 & set2)
    return (2 * intersection) / (len(set1) + len(set2)) if (len(set1) + len(set2)) > 0 else 0.0
def remove_duplicates_and_update_df(df: pd.DataFrame, col: str, threshold=0.75):
    unique_values = df[col].dropna().unique()
    unique_categories = []
    category_map = {}
    
    for category in unique_values:
        found_similar = False
        for unique_category in unique_categories:
            if dice_similarity(str(category), str(unique_category)) >= threshold:
                found_similar = True
                category_map[category] = unique_category
                break
        if not found_similar:
            unique_categories.append(category)
            category_map[category] = category
    
    df[col] = df[col].replace(category_map)
    return df
without_dups = dd.copy()
for col in dups_types:
    if dups_types.loc[0,col] == 'F3':
        without_dups = remove_duplicates_and_update_df(without_dups,col)

In [15]:
types_unmatched = dt.iloc[:,[1,2]]
types_unmatched = dict(zip(types_unmatched.iloc[:, 0], types_unmatched.iloc[:, 1]))
types_dict = dups_types.to_dict()
for key in types_dict.keys():
    types_dict[key] = types_dict[key][0]

merged_types = {**types_unmatched, **types_dict}
threshold = 0.1 * len(without_dups.columns)  # 10% of the columns must have non-null values

# Drop rows with more than 90% missing data
df_cleaned = without_dups.dropna(thresh=threshold)
pairs,df_sorted=sorted_neighborhood_g(df_cleaned,merged_types)

<ipython-input-11-ed3e8102ae89>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['blocker'] = (


0  window size :  3
1  window size :  3
2  window size :  3
3  window size :  3
4  window size :  3
5  window size :  3
6  window size :  3
7  window size :  3
8  window size :  3
9  window size :  3
10  window size :  3
11  window size :  3
12  window size :  3
13  window size :  3
14  window size :  3
15  window size :  3
16  window size :  3
17  window size :  3
18  window size :  3
19  window size :  3
20  window size :  3
21  window size :  3
22  window size :  3
23  window size :  3
24  window size :  3
25  window size :  3
26  window size :  3
27  window size :  3
28  window size :  3
29  window size :  3
30  window size :  3
31  window size :  3
32  window size :  3
33  window size :  3
34  window size :  3
35  window size :  3
36  window size :  3
37  window size :  3
38  window size :  3
39  window size :  3
40  window size :  3
41  window size :  3
42  window size :  3
43  window size :  3
44  window size :  3
45  window size :  3
46  window size :  3
47  window size :  3
48

In [18]:
import numpy as np
import random
def transitive_closure(relation):
    n = len(relation)
    for i in range(1, n):
        for j in range(i):
            if relation[i, j] == 1:
                for k in range(n):
                    if relation[j, k] == 1:
                        relation[i, k] = 1
    for i in range(n - 1):
        for j in range(i + 1, n):  # j from i+1 to n
            if relation[i, j] == 1:
                for k in range(n):
                    if relation[j, k] == 1:
                        relation[i, k] = 1
    return relation
def resolve_dups(df: pd.DataFrame, dups: list) -> pd.DataFrame:
    df['missing_values'] = df.isna().sum(axis=1)
    rows_to_drop = []
    for dup_pair in dups:
        row1, row2 = dup_pair
        if row1 not in df.index or row2 not in df.index:
            print(f"Skipping pair ({row1}, {row2}) as one of the rows does not exist.")
            continue
        missing_row1 = df.loc[row1, 'missing_values']
        missing_row2 = df.loc[row2, 'missing_values']
        if missing_row1 < missing_row2:
            rows_to_drop.append(row2)
        elif missing_row1 > missing_row2:
            rows_to_drop.append(row1)
        else:
            rows_to_drop.append(random.choice([row1, row2]))
    cleaned_df = df.drop(rows_to_drop, axis=0, errors='ignore')
    cleaned_df = cleaned_df.drop(columns=['missing_values'])
    return cleaned_df
def get_other_dups(transitive_closure_matrix: np.ndarray, tuples_list: list) -> list:
    dup_list = tuples_list
    for i in range(len(transitive_closure_matrix)):
        for j in range(len(transitive_closure_matrix)):
            if i == j:
                continue
            elif transitive_closure_matrix[i, j] == 1 and (i, j) not in tuples_list:
                dup_list.append((i, j))
    return dup_list
def fill_dup_matrix(dup_tuples: list, length: int):
    dup_mat = np.eye(length)
    for i, j in dup_tuples:
        dup_mat[i, j] = 1
    return dup_mat
final_df = resolve_dups(df_sorted, get_other_dups(
                    transitive_closure(fill_dup_matrix(pairs, len(df_sorted))),
                    pairs))

In [41]:
for pair in pairs:
   if compare(df_sorted,pair[0],pair[1],merged_types) > 0.95:
        print(pair)
#70% precision

(1214, 1216)
(1516, 1517)
(1994, 1995)
(3207, 3208)
(3383, 3384)
(4027, 4028)
(4489, 4490)


In [42]:
index_pairs = [(1214, 1216), (1516, 1517), (1994, 1995), 
               (3207, 3208), (3383, 3384), (4027, 4028), (4489, 4490)]
indices = [idx for pair in index_pairs for idx in pair]
filtered_rows = df_sorted.iloc[indices]
filtered_rows

,long_covid_post_covid_patient_timestamp,alter,groe_e,geschlecht,impfung_01_wirkstoff,impfung_02_wirkstoff,impfung_03_wirkstoff,impfung_04_wirkstoff,impfung_01_charge,impfung_02_charge,...,gpcr___4,gpcr___5,gpcr___6,gpcr___7,gpcr___8,gpcr___9,gpcr___10,gpcr___11,gpcr___12,gpcr___13
5046,2022-09-01 10:49:28,39.0,175.0,Männlich,Biontech/Pfizer,Moderna,keine,keine,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4646,2022-07-31 11:00:47,39.0,175.0,Männlich,Biontech/Pfizer,Moderna,keine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4938,2022-08-19 09:47:27,39.0,175.0,Weiblich,Biontech/Pfizer,Biontech/Pfizer,keine,keine,EX8680,FD7958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4939,2022-08-19 09:49:14,39.0,175.0,Weiblich,Biontech/Pfizer,Biontech/Pfizer,keine,keine,EX8680,FD7958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4883,2022-08-16 13:01:36,39.0,155.0,Weiblich,Moderna,Moderna,Moderna,NaN,30000489,3001651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4882,2022-08-16 12:50:46,39.0,155.0,Weiblich,Moderna,Moderna,Moderna,NaN,30000489,3001651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4958,[not completed],29.0,175.0,Männlich,Biontech/Pfizer,Biontech/Pfizer,keine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4959,[not completed],29.0,175.0,Männlich,Biontech/Pfizer,Biontech/Pfizer,keine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4890,[not completed],29.0,165.0,Weiblich,Biontech/Pfizer,Biontech/Pfizer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4891,[not completed],29.0,165.0,Weiblich,Biontech/Pfizer,Biontech/Pfizer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
